In [1]:
import hashlib
import progressbar
import json
print(hashlib.algorithms_available)

{'shake_256', 'whirlpool', 'sha3_224', 'shake256', 'sha3-384', 'shake_128', 'sha512-256', 'md4', 'sha512-224', 'sm3', 'blake2b512', 'blake2s256', 'md5-sha1', 'md5', 'blake2b', 'sha512', 'sha3-256', 'mdc2', 'sha256', 'sha3_384', 'blake2s', 'ripemd160', 'sha3_512', 'sha1', 'sha3-224', 'sha3-512', 'sha224', 'sha3_256', 'sha384', 'shake128'}


In [25]:
len(hashlib.md5_sha1("vkiril2".encode('utf-8')).hexdigest())

AttributeError: module 'hashlib' has no attribute 'md5_sha1'

In [2]:
def getDictionary(filename):
    return [line[:-1] for line in open(filename, "r")]
# getDictionary("files/dictionary.txt")

In [3]:
# Returns text with salt added
def salt(text):
    return [text + str(i).zfill(5) for i in range(100000)]
# salt("mike")

In [4]:
# Returns text under caesar cipher
def caesar(text):
    return ["".join(chr((ord(char) - 97 + shift) % 26 + 97) if char.isalpha() else char for char in text) for shift in range(26)]
# caesar("abcd12")

In [5]:
# Returns text in basic leet representation
def leet(text):
    chars = {"a":"4","e":"3","g":"6","i":"1","o":"0","s":"5","t":"7","z":"2"}
    return "".join(chars[char.lower()] if char in chars else char for char in text)
# leet("letmeinnow")

In [6]:
# Returns hashes of text
def hashed(text):
    func = [hashlib.md5, hashlib.sha1, hashlib.sha256, hashlib.sha512]
    return [f(text.encode('utf-8')).hexdigest() for f in func]
# hashed("lol")

In [7]:
# print(hashlib.md5("matrix123".encode('utf-8')).hexdigest())
# print("2224e8c5299874a5fe44a8de2f31c94e")
# print()
# print(hashlib.sha256("snowball138426".encode('utf-8')).hexdigest())
# print("e90829e04a20e9b49a4178f65b9a1a725191bd99fbd8071fa9cb6ee2552f2bc5")
# print()
# print(hashlib.sha512("tnynqevry".encode('utf-8')).hexdigest())
# print("6de664f06c446616eb374e1ba109a24b9172929162937831b18d7812fc76e1a3a8c9f4d1443d300b1b6a0419e80572ab0d5788f5faf28c869e06c84fe0242d96")
# print()
# print(hashlib.sha1("r0u5534u".encode('utf-8')).hexdigest())
# print("21d5c27a7e06317d1d02e0135a76454971e2c6d2")
# print()
# print(hashlib.sha256("josephine".encode('utf-8')).hexdigest())
# print("aec0e78824de1205cf98384780983c6f2b7c03f8a6e5036a508ece76f1b3c5a7")
# print()
# print(hashlib.sha512("pumpkin1".encode('utf-8')).hexdigest())
# print("fa5da9b79da364f5aea54b2b9cdedaa4fff6f522be7f1e2b5f13f399bd1c94ca059220d7fd1fed2a31a60068c2d13007da7bf987b1623a7fc0ada0390fafb584")
# print()
# print(hashlib.md5("".encode('utf-8')).hexdigest())
# print("dfa1339508e1b702a4588a3209a8e0ec")
# print()

In [8]:
#
# Accepts map of users and their hashed passwords along with dictionary filename
# Tries all possible combinations of all transformations but salt firstly
# Then looks for a salted password
#
def password_cracker(toFind, filename):
    # Map of hash size to its corresponding function
    hashMap = {32:hashlib.md5, 40:hashlib.sha1, 64:hashlib.sha256, 128:hashlib.sha512}
    found = {}
    error = {}
    
    # Handles the case if hashed password has len which is not defined in hashMap above
    for user in list(toFind):
        if not hashMap.get(len(toFind[user])):
            print("! Have no function defined for hash %s" % toFind[user])
            error[user] = toFind[user]
            del toFind[user]

    # Lets try all but salt
    for pwd in progressbar.progressbar(getDictionary(filename), prefix='All but salt: '):
        for pwdLeet in [pwd, leet(pwd)]:
            for pwdCaesar in caesar(pwdLeet):
                for user in list(toFind):
                    f = hashMap.get(len(toFind[user]))
                    finalPwd = pwdCaesar
                    if f(finalPwd.encode('utf-8')).hexdigest() == toFind[user]:
                        print("Found password for %s : %s" % (user, finalPwd))
                        found[user] = {
                            "password":finalPwd, 
                            "hashFunc":f.__name__, 
                            "seq":"%s -leet> %s -caesar> %s" % (pwd, pwdLeet, pwdCaesar)
                        }
                        del toFind[user]
                        
    # Salted password
#     for pwd in progressbar.progressbar(getDictionary(filename), prefix='Salt: '):
#         for pwdSalted in salt(pwd):
#             for user in list(toFind):
#                 f = hashMap.get(len(toFind[user]))
#                 if f(pwdSalted.encode('utf-8')).hexdigest() == toFind[user]:
#                     print("Found password for %s : %s" % (user, pwdSalted))
#                     found[user] = {
#                         "password":pwdSalted, 
#                         "hashFunc":f.__name__, 
#                         "seq":"%s -salt> %s" % (pwd, pwdSalted)
#                     }
#                     del toFind[user]
                
    # Print results
    print("\n----------------")
    print("Found passwords:")
    print(json.dumps(found, indent=2))
    
    print("\nErrors in hash length:")
    print(json.dumps(error, indent=2))
    
    print("\nNot found:")
    print(json.dumps(toFind, indent=2))
        

In [153]:
filename = "files/dictionary.txt"
toFind = {
    "user1":"2224e8c5299874a5fe44a8de2f31c94e",
    "user2":"e90829e04a20e9b49a4178f65b9a1a725191bd99fbd8071fa9cb6ee2552f2bc5",
    "user3":"6de664f06c446616eb374e1ba109a24b9172929162937831b18d7812fc76e1a3a8c9f4d1443d300b1b6a0419e80572ab0d5788f5faf28c869e06c84fe0242d96",
    "user4":"21d5c27a7e06317d1d02e0135a76454971e2c6d2",
    "user5":"aec0e78824de1205cf98384780983c6f2b7c03f8a6e5036a508ece76f1b3c5a7",
    "user6":"fa5da9b79da364f5aea54b2b9cdedaa4fff6f522be7f1e2b5f13f399bd1c94ca059220d7fd1fed2a31a60068c2d13007da7bf987b1623a7fc0ada0390fafb584",
    "user7":"dfa1339508e1b702a4588a3209a8e0ec"
}
password_cracker(toFind, filename);

All but salt:  18% (911 of 4797) |#      | Elapsed Time: 0:00:00 ETA:   0:00:03

Found password for user5 : josephine


All but salt:  35% (1701 of 4797) |##    | Elapsed Time: 0:00:01 ETA:   0:00:02

Found password for user6 : pumpkin1


All but salt:  58% (2794 of 4797) |###   | Elapsed Time: 0:00:01 ETA:   0:00:01

...

Found password for user1 : matrix123


Salt:  81% (3916 of 4797) |###########   | Elapsed Time: 0:24:45 ETA:   0:05:04

Found password for user2 : 0000111


Salt: 100% (4797 of 4797) |##############| Elapsed Time: 0:27:49 Time:  0:27:49



----------------
Found passwords:
{
  "user5": {
    "password": "josephine",
    "hashFunc": "openssl_sha256",
    "seq": "josephine -leet> josephine -caesar> josephine"
  },
  "user6": {
    "password": "pumpkin1",
    "hashFunc": "openssl_sha512",
    "seq": "pumpkin1 -leet> pumpkin1 -caesar> pumpkin1"
  },
  "user3": {
    "password": "tnynqevry",
    "hashFunc": "openssl_sha512",
    "seq": "galadriel -leet> galadriel -caesar> tnynqevry"
  },
  "user4": {
    "password": "r0u5534u",
    "hashFunc": "openssl_sha1",
    "seq": "rousseau -leet> r0u5534u -caesar> r0u5534u"
  },
  "user1": {
    "password": "matrix123",
    "hashFunc": "openssl_md5",
    "seq": "matrix123 -leet> matrix123 -caesar> matrix123"
  },
  "user2": {
    "password": "snowball138426",
    "hashFunc": "openssl_sha256",
    "seq": "snowball1 -salt> snowball138426"
  }
}

Errors in hash length:
{}

Not found:
{
  "user7": "dfa1339508e1b702a4588a3209a8e0ec"
}
